In [31]:
# Install library yang dibutuhkan
!pip install git+https://github.com/ibm-granite-community/utils \
    "langchain_community<0.3.0" \
    replicate pyngrok flask

  Cloning https://github.com/ibm-granite-community/utils to /tmp/pip-req-build-55ysbznk
  Running command git clone --filter=blob:none --quiet https://github.com/ibm-granite-community/utils /tmp/pip-req-build-55ysbznk
  Resolved https://github.com/ibm-granite-community/utils to commit 1514191fbbc4605ed4fdfdcb448f2ee41477058f
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [32]:
from ibm_granite_community.notebook_utils import get_env_var
from langchain_community.llms import Replicate

# Inisialisasi model Granite via Replicate
model = Replicate(
    model="ibm-granite/granite-3.3-8b-instruct",
    replicate_api_token=get_env_var("REPLICATE_API_TOKEN"),  # Harus sudah diset
    model_kwargs={"max_tokens": 512, "temperature": 0.2},
)


In [33]:
def zeroshot_prompt_diet(data):
    return f"""
    Saya pengguna aplikasi diet bernama Kaloriku. Hari ini saya telah mengonsumsi {data['kalori_masuk']}kkal dan membakar {data['kalori_terbakar']}kkal melalui aktivitas.
    Target saya adalah defisit {data['target_defisit']}kkal.
    Tolong berikan saran makanan yang harus saya kurangi atau aktivitas tambahan yang bisa saya lakukan agar target defisit tercapai.
    Berikan jawaban ringkas dan mudah dipahami.
    """


In [34]:
from flask import Flask, request, jsonify
import threading

app = Flask(__name__)

@app.route("/saran-ai", methods=["POST"])
def saran_ai():
    data = request.json
    prompt = zeroshot_prompt_diet(data)
    result = model.invoke(prompt)
    return jsonify({"saran": result})

def run_flask():
    app.run(host="0.0.0.0", port=5000)

# Jalankan Flask di thread terpisah
thread = threading.Thread(target=run_flask)
thread.start()


 * Serving Flask app '__main__'
 * Debug mode: off


In [37]:
from pyngrok import ngrok

# Set token dan koneksi
ngrok.set_auth_token("2zLwnLDph938yfIobq2o8Ue8aZS_2GPedgYx8NDYQz3KHv4sV")
tunnel = ngrok.connect(5000)
public_url = tunnel.public_url  # 🔥 ambil string URL-nya saja

print("Public URL:", public_url)

Public URL: https://9683-34-168-218-235.ngrok-free.app


In [38]:
import requests

data = {
    "kalori_masuk": 1800,
    "kalori_terbakar": 300,
    "target_defisit": 500
}

response = requests.post(f"{public_url}/saran-ai", json=data)
print("Saran AI:\n", response.json()["saran"])


INFO:werkzeug:127.0.0.1 - - [03/Jul/2025 07:11:20] "POST /saran-ai HTTP/1.1" 200 -


Saran AI:
 Saya memahami bahwa Anda menggunakan aplikasi diet bernama Kaloriku dan ingin mencapai defisit kalori sebesar 500 kkal. Anda telah mengonsumsi 1800 kcal dan membakar 300 kcal melalui aktivitas. 

Untuk mencapai target defisit 500 kcal, Anda perlu mengurangi kalori masuk atau meningkatkan kalori terbakar sebanyak 500 kcal. Berikut adalah beberapa saran untuk Anda:

1. Mengurangi kalori masuk:
   - Pilihlah makanan yang lebih ringan dan rendah kalori, seperti sayuran, buah, dan protein yang diproses sederhana (misalnya, daging putih, ikan, tofu).
   - Hindari makanan yang tinggi kalori, seperti makanan yang berkembang, minyak, dan makanan yang tinggi karbohidrat.
   - Minimalkan konsumsi makanan dan minuman yang tinggi kalori, seperti minuman beralkohol dan minuman segar yang tidak diperiksa.

2. Meningkatkan kalori terbakar:
   - Tambahkan aktivitas fisik lebih intensif, seperti olahraga ringan hingga berat seperti berjoging, berenang, atau melakukan latihan HIIT (High-Intens